In [1]:
!pip install fastapi
!pip install uvicorn


     -------------------------------------- 92.1/92.1 kB 750.7 kB/s eta 0:00:00
     ------------------------------------ 395.2/395.2 kB 769.9 kB/s eta 0:00:00
     -------------------------------------- 71.5/71.5 kB 785.1 kB/s eta 0:00:00
     ---------------------------------------- 1.9/1.9 MB 1.5 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.3.0
    Uninstalling typing_extensions-4.3.0:
      Successfully uninstalled typing_extensions-4.3.0
     -------------------------------------- 60.8/60.8 kB 802.7 kB/s eta 0:00:00
     ---------------------------------------- 58.3/58.3 kB 3.0 MB/s eta 0:00:00


In [7]:
%%capture
!mkdir app
!mkdir app/models
!mkdir app/routers
!mkdir app/schemas
!mkdir app/middleware
!mkdir tests

# Create files within the directories
!touch app/models/__init__.py
!touch app/models/assets.py
!touch app/routers/__init__.py
!touch app/routers/assets.py
!touch app/schemas/__init__.py
!touch app/schemas/assets.py
!touch app/main.py
!touch requirements.txt
!touch README.md

In [8]:
%%writefile app/main.py
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from app.routers import assets, performance_metrics

app = FastAPI()

# Enable CORS for all origins
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

app.include_router(assets.router)
app.include_router(performance_metrics.router)


Overwriting app/main.py


In [10]:
%%writefile app/routers/assets.py
from fastapi import APIRouter, HTTPException
from typing import List
from app.models import Asset
from app.schemas import AssetCreate

router = APIRouter()

@router.get("/assets/", response_model=List[Asset])
async def read_assets():
    # Implement your logic to fetch assets
    pass

# Implement other CRUD operations for assets



Writing app/routers/assets.py


FileNotFoundError: [Errno 2] No such file or directory: 'app/routers/assets.py'

In [6]:
%%writefile app/schemas/assets.py
from pydantic import BaseModel
from typing import Optional

class Asset(BaseModel):
    asset_id: str
    asset_name: str
    asset_type: str
    location: str
    purchase_date: str
    initial_cost: float
    operational_status: str

class AssetCreate(Asset):
    pass


Writing app/schemas/assets.py


FileNotFoundError: [Errno 2] No such file or directory: 'app/schemas/assets.py'

In [11]:
%%writefile app/models.py
from pydantic import BaseModel

class Asset(BaseModel):
    asset_id: str
    asset_name: str
    asset_type: str
    location: str
    purchase_date: str
    initial_cost: float
    operational_status: str


Writing app/models.py


In [14]:
%%writefile app/schemas/assets.py
from pydantic import BaseModel
from typing import Optional

class Asset(BaseModel):
    asset_id: str
    asset_name: str
    asset: str
    location: str
    purchase_date: str
    initial_cost: float
    operational_status: str

class AssetCreate(Asset):
    pass



Writing app/schemas/assets.py


FileNotFoundError: [Errno 2] No such file or directory: 'app/schemas/assets.py'

In [15]:
%%writefile app/routers/assets.py
from fastapi import APIRouter, HTTPException
from typing import List
from .schemas.assets import Asset, AssetCreate
from pymongo import MongoClient

router = APIRouter()

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["asset_performance"]
collection = db["assets"]

@router.post("/assets/", response_model=Asset)
async def create_asset(asset: AssetCreate):
    # Insert new asset into MongoDB
 new_asset = dict(asset.dict())
    result = collection.insert_one(new_asset)
    # Return the created asset
    return asset

@router.get("/assets/{asset_id}", response_model=Asset)
async def read_asset(asset_id: str):
    Fetch asset from MongoDB using asset_id
    asset_data = collection.find_one({"asset_id": asset_id})
    if asset_data:
        return asset_data
    raise HTTPException(status_code=404, detail="Asset not")

@router.put("/assets/{asset_id}", response_model=Asset)
async def update_asset(asset_id: str, asset: Asset):
    # Update the asset in MongoDB based on asset_id
    updated_asset = dict(asset.dict())
    result = collection.update_one({"asset_id": asset_id}, {"$set": updated_asset    if result.modified_count == 1:
        return asset
    raise HTTPException(status_code=404, detail="Asset not found")

@router.delete("/assets/{asset_id}", response_model=str)
async def delete_asset(asset_id: str):
    # Delete asset MongoDB
    result = collection.delete_one({"asset_id":_id})
    if result.deleted_count == 1:
        return "Asset deleted successfully"
    raise HTTPException(status_code=404, detail="Asset not found")

@router.get("/assets/", response_model=List[Asset])
async def read_assets():
    # Fetch all assets from MongoDB
    assets_data = list(collection.find({}))
    return assets_data



Writing app/routers/assets.py


FileNotFoundError: [Errno 2] No such file or directory: 'app/routers/assets.py'

In [19]:
%%writefile app/main.py
from fastapi import FastAPI, HTTPException, Depends
from fastapi.middleware.cors import CORSMiddleware
from fastapi.security import OAuth2PasswordBearer, OAuth2PasswordRequestForm
from datetime import timedelta, datetime
import jwt
from .routers import assets, performance_metrics

app = FastAPI()

# Enable CORS for all origins
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
   _methods=["*"],
   _headers=[""],
)

# Secret key to sign JWT token
SECRET_KEY = "your_secret_key"
ALGORITHM = "HS256"
ACCESS_TOKEN_EXPIREUTES = 30

# Dummy user for demonstration
users_db =    "demo_user": {
        "username": "demo_user",
        "password": "demopassword",
    }

    }
 Function to create access token using JWT
def create_access_token(data: dict, expires_delta: timedelta):
    to_encode = data.copy()
    expire = datetime.utcnow() + expires_delta
    to_encode.update({"exp": expire})
    encoded_jwt = jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)
    return encoded_jwt

# Endpoint to login and generate access token
@app.post("/token")
async def login_for_access_token(form_data: OAuth2PasswordRequestForm = Depends()):
    user = form_data.username
    password = form_data.password
    if user in users_db password == users_db[user]["password"]:
        access_token_expires = timedelta(minutes=ACCESS_TOKEN_EXPIRE_MINUTES)
        access_token = create_access_token(
            data={"sub": user}, expires_delta=access_token_expires
        )
        return {"access_token": access_token, "token_type": "bearer"}
    raise HTTPException(status_code=400, detail="Incorrect username or password")

app.include_router(assets.router, prefix="/assets", tags=["assets"])
app.include_router(performance_metrics.router, prefix="/metrics", tags=["metrics"])


Overwriting app/main.py


In [20]:
# Import required libraries for MongoDB aggregation
from pymongo import MongoClient
from fastapi import APIRouter, HTTPException
from typing import List

router = APIRouter()

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["asset_performance"]
collection = db["performance_metrics"]

# Endpoint to calculate average downtime across all
@router.get("/metrics/average_downtime/", response_model=float)
async def calculate_average_downtime():
    average_downtime = collection.aggregate([
        {"$group": {"_id": None, "avg_downtime": {"$avg": "$downtime"}}}
    ])
    result = listaverage_downtime)
    if result:
        return result[0]["avg_downtime"]
   :
        return 0.0

# Endpoint to sum total maintenance costs within a selected time frame
@router.get("/metrics/total_maintenance_costs/", response_model)
async def calculate_total_maintenance_costs(start_date: str, end_date: str):
    totalaintenance_costs = collection.aggregate([
        {"$match": {"$and": [{"date": {"$gte": start_date}}, {"date": {"$lte": end_date}}]}},
        {"$": {"_id": None, "total_costs": {"$sum": "$_costs"}}}
    ])
    result = list(total_maintenance_costs)
    if result:
        return result[0]["total_costs"]
    else:
        return 0.0# Endpoint to identify assets with high failure rates
@router.get("/metrics/high_failure_assets/", response_model=List[str])
async def identify_high_failure_assets(threshold: int = 10):
    high_failure_assets =.aggregate([
        {"$match": {"failure_rate": {"$gte": threshold}}},
        {"$": {"_id": "$asset_id"}}
    ])
    result = list(high_failure_assets)
   _failure_asset_ids = [asset["_id"] for asset in result]
    return high_failure_asset_ids


SyntaxError: unmatched ')' (2550109692.py, line 19)

In [23]:
 !pip install httpx
# tests/test_routers/test_assets.py
from fastapi.testclient import TestClient
from ..app.main import app

client = TestClient(app)

def test_read_assets():
    response = client.get("/assets/")
    assert response.status_code == 200

def test_create_asset():
    response = client.post("/assets/", json={"asset_id": "1", "asset_name": "Sample Asset"})
    assert response.status_code == 200
    asset = response.json()
    assert asset["asset_name"] == "Sample Asset"

def test_read_asset():
    response = client.get("/assets/1")
    assert response.status_code == 200
    asset = response.json()
    assert asset["asset_id"] == "1"

def test_update_asset():
    response = client.put("/assets/1", json={"asset_id": "1", "asset_name": "Updated Asset"})
    assert response.status_code == 200
    asset = response.json()
    assert asset["asset_name"] == "Updated Asset"

def test_delete_asset():
    response = client.delete("/assets/1")
    assert response.status_code == 200
    assert response.text == "Asset deleted successfully"


     -------------------------------------- 75.6/75.6 kB 837.8 kB/s eta 0:00:00
     ---------------------------------------- 77.8/77.8 kB 2.1 MB/s eta 0:00:00


ImportError: attempted relative import with no known parent package